In [ ]:
import numpy as np
from math import sqrt

## 付録E　床下に空気を供給する場合の床下温度等の算定方法 / Appendix E Calculation method of underfloor temperature etc. in the case of supplying air under the floor

### E.1 記号及び単位 / Symbol and unit

省略

Omission

### E.2 床下温度 / Underfloor temperature

$$
\begin{equation*}
   \theta_{uf,d,t} = \frac{\rho_{air} c_{p_{air}} V_{sa,d,t,A} \theta_{sa,d,t} + \left( \sum_{i=1}^{12} U_s A_{s,ufvnt,i} H_{d,t,i}^* \theta_{d,t,i}^* + \varphi L_{uf} \theta_{ex,d,t} + \frac{A_{s,ufvnt,A}}{R_g} \frac{\sum_{m=1}^{M} \theta'_{g,surf,A,m.d,t} + \theta_{g,avg}}{1 + \frac{\phi_{A,0}}{R_g}} \right) \times 3.6}{\rho_{air} c_{p_{air}} V_{sa,d,t,A} + \left( \sum_{i=1}^{12} U_s A_{s,ufvnt,i} H_{d,t,i}^* + \varphi L_{uf} + \frac{A_{s,ufvnt,A}}{R_g} \frac{1}{1 + \frac{\phi_{A,0}}{R_g}} \right) \times 3.6}
\end{equation*}
$$

In [ ]:
# 床下温度及び地盤またはそれを覆う基礎の表面温度 (℃) (1)(9)
def calc_Theta(region, A_A, A_MR, A_OR, Q, r_A_ufvnt, underfloor_insulation, Theta_sa_d_t, Theta_ex_d_t,
               V_sa_d_t_A, H_OR_C,
               L_dash_H_R_d_t=None,
               L_dash_CS_R_d_t=None):
    """
    :return: 床下温度、地盤またはそれを覆う基礎の表面温度 (℃)
    """
    # 地盤またはそれを覆う基礎の表面熱伝達抵抗 ((m2・K)/W)
    R_g = 0.15

    # 吸熱応答係数の初項
    Phi_A_0 = 0.025504994

    # 空気密度[kg/m3]
    ro_air = get_ro_air()

    # 空気の比熱[kJ/kgK]
    c_p_air = get_c_p_air()

    # 床の熱貫流率[W/m2K]
    U_s = get_U_s()

    # 指数項の総数
    M = 10

    # 地盤の不易層温度 (℃) (10)
    Theta_g_avg = get_Theta_g_avg(Theta_ex_d_t)

    # 作業領域の確保
    Theta_uf_d_t = np.zeros(24 * 365)
    Theta_g_surf_d_t = np.zeros(24 * 365)
    Theta_dash_g_surf_A_m = np.zeros(M)

    # 初期値の設定
    Theta_uf_prev = 0.0
    Theta_g_surf_prev = 0.0
    Theta_dash_g_surf_A_m_prev = [0.0] * M

    # ----- 助走計算 -----

    # 助走計算用床下温度
    Theta_uf_runup = get_Theta_uf_d_t_runup(underfloor_insulation, Theta_ex_d_t)

    Theta_in_H = 20
    Theta_in_C = 27

    # 指数項mにおける吸熱応答係数
    phi_1_A_m = np.array([get_phi_1_A_m(m) for m in range(1, M + 1)])

    # 指数項mにおける公比r_m
    r_m = np.array([get_r_m(m) for m in range(1, M + 1)])

    for dt in range(24 * 365):
        # 当該住宅の床下温度 (13)
        Theta_uf = Theta_uf_runup[dt]

        # 日付dの時刻t-1における地盤またはそれを覆う基礎の表面熱流 (W/m2) (12)
        q_g_prev = 1.0 / R_g * (Theta_uf_prev - Theta_g_surf_prev)

        # 日付dの時刻tにおける指数項mの吸熱応答の項別成分 (℃) (11)
        Theta_dash_g_surf_A_m = phi_1_A_m * q_g_prev + r_m * Theta_dash_g_surf_A_m_prev

        # 地盤またはそれを覆う基礎の表面温度 (℃) (9)
        Theta_g_surf = ((Phi_A_0 / R_g) * Theta_uf + np.sum(Theta_dash_g_surf_A_m) + Theta_g_avg) \
                       / (1.0 + (Phi_A_0 / R_g))

        # 次時刻へ値を保存
        Theta_uf_prev = Theta_uf
        Theta_g_surf_prev = Theta_g_surf
        Theta_dash_g_surf_A_m_prev = Theta_dash_g_surf_A_m

    # ----- 本番計算 -----

    if r_A_ufvnt is None:
        r_A_ufvnt = 0

    # 当該住戸の暖冷房区画iの空気を供給する床下空間に接する床の面積(m2) (7)
    A_s_ufvnt = [get_A_s_ufvnt_i(i, r_A_ufvnt, A_A, A_MR, A_OR) for i in range(1, 13)]

    # 当該住戸の空気を供給する床下空間に接する床の面積の合計 (m2) (8)
    A_s_ufvnt_A = get_A_s_ufvnt_A(r_A_ufvnt, A_A, A_MR, A_OR)

    # 当該住戸の外気を導入する床下空間の基礎外周長さ (6)
    L_uf = get_L_uf(A_s_ufvnt_A)

    H_floor = 0.7

    # 基礎の線熱貫流率 (W/mK) (5)
    phi = get_phi(region, Q)

    Theta_star = np.zeros(12)
    H_star = np.zeros(12)

    for dt in range(24 * 365):
        # 日付dの時刻t-1における地盤またはそれを覆う基礎の表面熱流 (W/m2) (12)
        q_g_prev = 1.0 / R_g * (Theta_uf_prev - Theta_g_surf_prev)

        # 日付dの時刻tにおける指数項mの吸熱応答の項別成分 (℃) (11)
        Theta_dash_g_surf_A_m = phi_1_A_m * q_g_prev + r_m * Theta_dash_g_surf_A_m_prev

        # 参照空気温度及び参照温度差係数 (2)(3)
        # TODO: ここをベクトル化する
        if H_OR_C == 'H':
            for i in range(1, 13):
                if L_dash_H_R_d_t[i - 1][dt] <= 0 and Theta_ex_d_t[dt] < Theta_in_H:
                    Theta_star[i - 1] = Theta_in_H
                    H_star[i - 1] = 1
                elif L_dash_H_R_d_t[i - 1][dt] <= 0 and Theta_in_H <= Theta_ex_d_t[dt] and Theta_ex_d_t[
                    dt] <= Theta_in_C:
                    Theta_star[i - 1] = Theta_ex_d_t[dt]
                    H_star[i - 1] = H_floor
                elif L_dash_H_R_d_t[i - 1][dt] <= 0 and Theta_in_C <= Theta_ex_d_t[dt]:
                    Theta_star[i - 1] = Theta_in_C
                    H_star[i - 1] = 1
                elif 0 < L_dash_H_R_d_t[i - 1][dt]:
                    Theta_star[i - 1] = Theta_in_H
                    H_star[i - 1] = 1
                else:
                    raise ValueError(L_dash_H_R_d_t[i - 1][dt])
        elif H_OR_C == 'C':
            for i in range(1, 13):
                if L_dash_CS_R_d_t[i - 1][dt] <= 0 and Theta_ex_d_t[dt] < Theta_in_H:
                    Theta_star[i - 1] = Theta_in_H
                    H_star[i - 1] = 1
                elif L_dash_CS_R_d_t[i - 1][dt] <= 0 and Theta_in_H <= Theta_ex_d_t[dt] and Theta_ex_d_t[
                    dt] <= Theta_in_C:
                    Theta_star[i - 1] = Theta_ex_d_t[dt]
                    H_star[i - 1] = H_floor
                elif L_dash_CS_R_d_t[i - 1][dt] <= 0 and Theta_in_C < Theta_ex_d_t[dt]:
                    Theta_star[i - 1] = Theta_in_C
                    H_star[i - 1] = 1
                elif 0 < L_dash_CS_R_d_t[i - 1][dt]:
                    Theta_star[i - 1] = Theta_in_C
                    H_star[i - 1] = 1
                else:
                    raise ValueError(L_dash_CS_R_d_t[i - 1][dt])

        else:
            raise ValueError(H_OR_C)

        # 当該住宅の床下温度 (1)
        Theta_uf = (ro_air * c_p_air * V_sa_d_t_A[dt] * Theta_sa_d_t[dt] +
                    (sum([U_s * A_s_ufvnt[i - 1] * H_star[i - 1] * Theta_star[i - 1] for i in
                          range(1, 13)])
                     + phi * L_uf * Theta_ex_d_t[dt]
                     + (A_s_ufvnt_A / R_g)
                     * ((sum(Theta_dash_g_surf_A_m) + Theta_g_avg) / (1.0 + Phi_A_0 / R_g))) * 3.6) \
                   / (ro_air * c_p_air * V_sa_d_t_A[dt]
                      + (sum([U_s * A_s_ufvnt[i - 1] * H_star[i - 1] for i in range(1, 13)])
                         + phi * L_uf
                         + (A_s_ufvnt_A / R_g)
                         * (1.0 / (1.0 + Phi_A_0 / R_g))) * 3.6)

        # 地盤またはそれを覆う基礎の表面温度 (℃) (9)
        Theta_g_surf = ((Phi_A_0 / R_g) * Theta_uf + np.sum(Theta_dash_g_surf_A_m) + Theta_g_avg) \
                       / (1.0 + (Phi_A_0 / R_g))

        # 次時刻へ値を保存
        Theta_uf_prev = Theta_uf
        Theta_g_surf_prev = Theta_g_surf
        Theta_dash_g_surf_A_m_prev = Theta_dash_g_surf_A_m

        # 計算結果の保存
        Theta_uf_d_t[dt] = Theta_uf
        Theta_g_surf_d_t[dt] = Theta_g_surf

    return Theta_uf_d_t, Theta_g_surf_d_t

$\rho_{air}$： 空気の密度(kg/m<sup>3</sup>) / Air density   
$c_{p_{air}}$： 空気の比熱(kJ/(kg・K))  / Specific heat of air  
$H_{d,t,i}^*$：日付$d$の時刻$t$における暖冷房区画$i$の参照温度差係数(-) / Reference temperature difference coefficient of heating / cooling section $ i $ at time $ t $ of date $ d $  
$V_{sa,d,t,A}$：日付$d$の時刻$t$における1 時間当たりの床下空間へ供給する空気の風量の合計(m<sup>3</sup>/h) / Total volume of air supplied to the underfloor space per hour at time $ t $ on date $ d $  
$U_s$：床の熱貫流率(W/(m<sup>2</sup>・K)) / Heat transmission coefficient of floor  
$A_{s,ufvnt,i}$：当該住戸の暖冷房区画$i$の空気を供給する床下空間に接する床の面積(m<sup>2</sup>) / Floor area in contact with the underfloor space that supplies air for heating / cooling section $ i $ of the relevant dwelling unit  
$A_{s,ufvnt,A}$：当該住戸の空気を供給する床下空間に接する床の面積の合計(m<sup>2</sup>) / Total floor area in contact with the floor space that supplies the air of the dwelling unit  
$L_{uf}$：当該住戸の空気を供給する床下空間の基礎外周長さ(m) / Base perimeter length of the space under the floor which supplies the air of the dwelling unit concerned  
$M$：指数項の総数(-) / Total number of exponential terms  
$R_g$：地盤またはそれを覆う基礎の表面熱伝達抵抗((m<sup>2</sup>・K)/W) / Surface heat transfer resistance of ground or foundation covering it  
$\theta_{d,t,i}^*$：日付$d$の時刻$t$における暖冷房区画$i$の参照空気温度(℃) / Reference air temperature of heating / cooling compartment $ i $ at time $ t $ of date $ d $  
$\theta_{ex,d,t}$：日付$d$の時刻$t$における外気温度(℃) / Outside temperature at time $ t $ of date $ d $  
$\theta_{g,avg}$：地盤の不易層温度(℃) / Impossible layer temperature of the ground  
$\theta_{uf,d,t}$：日付$d$の時刻$t$における当該住戸の床下温度(℃) / Under floor temperature of the dwelling unit at time $ t $ of date $ d $  
$\theta_{sa,d,t}$：日付$d$の時刻$t$において床下空間へ供給する空気の温度(℃) / Temperature of air supplied to the underfloor space at time $ t $ of date $ d $  
$\theta'_{g,surf,A,m,d,t}$：日付$d$の時刻$t$における指数項$m$の吸熱応答の項別成分(℃) / Term component of endothermic response of exponent term $ m $ at time $ t $ of date $ d $  
$\varphi$：基礎の線熱貫流率(W/(m・K)) / Line heat penetration coefficient of foundation  
$\phi_{A,0}$：吸熱応答係数の初項((m<sup>2</sup>・K)/W) / First term of endothermic response coefficient  

空気の密度$\rho_{air}$は、1.20 kg/m<sup>3</sup>とする。空気の比熱$c_{p_{air}}$は、1.006 kJ/(kg・K)とする。床の熱貫流率$U_s$は、2.223W/(m<sup>2</sup>・K)とする。

The density of air $ \rho_ {air} $ is 1.20 kg / m<sup>3</sup>. The specific heat $ c_ {p_ {air}} $ of air is 1.006 kJ / (kg · K). The heat transmission coefficient $ U_s $ of the bed is 2.223 W / (m<sup>2</sup> · K).

In [ ]:
# 空気の密度 (kg/m3)
def get_ro_air():
    return 1.20


# 空気の比熱 (kJ/(kg・K))
def get_c_p_air():
    return 1.006


# 床の熱貫流率 (W/(m2・K))
def get_U_s():
    return 2.223

$L'_{H,R,d,t,i} \leq 0$ かつ $\theta_{ex,d,t} < \theta_{in,H}$ の場合： 

$L'_{H,R,d,t,i} \leq 0$ and $\theta_{ex,d,t} < \theta_{in,H}$:

$$\large
\begin{equation*}
\theta_{d,t,i}^* = \theta_{in,H}
\end{equation*}
$$

$$\large
\begin{equation*}
H_{d,t,i}^* = 1
\end{equation*}
$$

$L'_{H,R,d,t,i} \leq 0$ かつ $\theta_{in,H} \leq \theta_{ex,d,t} \leq \theta_{in,C}$ の場合： 

$L'_{H,R,d,t,i} \leq 0$ and $\theta_{in,H} \leq \theta_{ex,d,t} \leq \theta_{in,C}$:

$$\large
\begin{equation*}
\theta_{d,t,i}^* = \theta_{ex,d,t}
\end{equation*}
$$

$$\large
\begin{equation*}
H_{d,t,i}^* = H_{floor}
\end{equation*}
$$

$L'_{H,R,d,t,i} \leq 0$ かつ $\theta_{in,C} < \theta_{ex,d,t}$ の場合： 

For $L'_{H,R,d,t,i} \leq 0$ and $\theta_{in,C} < \theta_{ex,d,t}$:

$$\large
\begin{equation*}
\theta_{d,t,i}^* = \theta_{in,C}
\end{equation*}
$$

$$\large
\begin{equation*}
H_{d,t,i}^* = 1
\end{equation*}
$$

$0 < L'_{H,R,d,t,i}$ の場合： 

For $0 < L'_{H,R,d,t,i}$:

$$\large
\begin{equation*}
\theta_{d,t,i}^* = \theta_{in,H}
\end{equation*}
$$

$$\large
\begin{equation*}
H_{d,t,i}^* = 1
\end{equation*}
$$

$L'_{CS,R,d,t,i} \leq 0$ かつ $\theta_{ex,d,t} < \theta_{in,H}$ の場合： 

$L'_{CS,R,d,t,i} \leq 0$ and $\theta_{ex,d,t} < \theta_{in,H}$:

$$\large
\begin{equation*}
\theta_{d,t,i}^* = \theta_{in,H}
\end{equation*}
$$

$$\large
\begin{equation*}
H_{d,t,i}^* = 1
\end{equation*}
$$

$L'_{CS,R,d,t,i} \leq 0$ かつ $\theta_{in,H} \leq \theta_{ex,d,t} \leq \theta_{in,C}$ の場合： 

$L'_{CS,R,d,t,i} \leq 0$ and $\theta_{in,H} \leq \theta_{ex,d,t} \leq \theta_{in,C}$:

$$\large
\begin{equation*}
\theta_{d,t,i}^* = \theta_{ex,d,t}
\end{equation*}
$$

$$\large
\begin{equation*}
H_{d,t,i}^* = H_{floor}
\end{equation*}
$$

$L'_{CS,R,d,t,i} \leq 0$ かつ $\theta_{in,C} < \theta_{ex,d,t}$ の場合： 

For $L'_{CS,R,d,t,i} \leq 0$ and $\theta_{in,C} < \theta_{ex,d,t}$:

$$\large
\begin{equation*}
\theta_{d,t,i}^* = \theta_{in,C}
\end{equation*}
$$

$$\large
\begin{equation*}
H_{d,t,i}^* = 1
\end{equation*}
$$

$0 < L'_{CS,R,d,t,i}$ の場合： 

For $0 < L'_{CS,R,d,t,i}$:

$$\large
\begin{equation*}
\theta_{d,t,i}^* = \theta_{in,H}
\end{equation*}
$$

$$\large
\begin{equation*}
H_{d,t,i}^* = 1
\end{equation*}
$$

$H_{d,t,i}^*$：日付$d$の時刻$t$における暖冷房区画$i$の参照温度差係数(-) / Reference temperature difference coefficient of heating / cooling section $ i $ at time $ t $ of date $ d $  
$H_{floor}$：床の温度差係数(-) / Floor temperature difference coefficient  
$L'_{CS,R,d,t,i}$：日付$d$の時刻$t$における標準住戸の暖冷房区画$i$の負荷補正前の冷房顕熱負荷(MJ/h) / Cooling sensible heat load before load correction of heating / cooling section $ i $ of standard dwelling unit at time $ t $ of date $ d $  
$L'_{H,R,d,t,i}$：日付$d$の時刻$t$における標準住戸の暖冷房区画$i$の負荷補正前の暖房負荷(MJ/h) / Heating load before load compensation of heating / cooling section $ i $ of standard dwelling unit at time $ t $ of date $ d $  
$\theta_{ex,d,t}$：日付$d$の時刻$t$における外気温度(℃) / Outside temperature at time $ t $ of date $ d $  
$\theta_{in,C}$：冷房時の室温(℃) / Room temperature at cooling  
$\theta_{in,H}$：暖房時の室温(℃) / Room temperature when heating  
$\theta_{d,t,i}^*$：日付$d$の時刻$t$における暖冷房区画$i$の参照空気温度(℃) / Reference air temperature of heating / cooling compartment $ i $ at time $ t $ of date $ d $  

床の温度差係数$H_{floor}$は、0.7 とする。暖房時の室温$\theta_{in,H}$は、20℃とする。冷房時の室温$\theta_{in,C}$は、27℃とする。 

The floor temperature difference coefficient $ H_ {floor} $ is 0.7. The room temperature $ \theta_ {in, H} $ at heating is assumed to be 20°C. The room temperature $\theta_ {in, C} $ at the time of cooling is 27°C.

### E.3 床の熱貫流率及び基礎の線熱貫流率 / Heat transmission coefficient of floor and linear heat transmission coefficient of foundation

暖冷房負荷計算時に想定した床の熱貫流率$U_{s,vert}$が2.223 W/(m<sup>2</sup>・K)を上回る場合には、2.223 W/(m2・K)とする。また、8 地域においては、暖冷房負荷計算時に想定した床の熱貫流率$U_{s,vert}$は、当該住戸の熱損失係数$Q$に係わらず2.223W/(m<sup>2</sup>・K)とする。 

If the floor heat transmission coefficient $ U_ {s, vert} $ assumed at the time of heating and cooling load calculation exceeds 2.223W/(m<sup>2</sup>・K), it is set to 2.223 W/(m2・K). Also, in the eight regions, the floor heat transmission coefficient $ U_ {s, vert} $ assumed at the time of heating and cooling load calculation is set to 2.223W/(m<sup>2</sup>・K) regardless of the heat loss coefficient $ Q $ of the relevant dwelling unit.

$$
\large
U_{s,vert} = \left\{
\begin{array}{ll}
\frac{Q - Q_{1}}{Q_{0} - Q_{1}} \times U_{s,R,Q_{0}} + \frac{Q - Q_{0}}{Q_{1} - Q_{0}} \times U_{s,R,Q_{1}} & (Q > Q_{1} の場合)\\
\frac{Q - Q_{2}}{Q_{1} - Q_{2}} \times U_{s,R,Q_{1}} + \frac{Q - Q_{1}}{Q_{2} - Q_{1}} \times U_{s,R,Q_{2}} & (Q_{1} \geq Q > Q_{2} の場合)\\
\frac{Q - Q_{3}}{Q_{2} - Q_{3}} \times U_{s,R,Q_{2}} + \frac{Q - Q_{2}}{Q_{3} - Q_{2}} \times U_{s,R,Q_{3}} & (Q_{2} \geq Q > Q_{3} の場合)\\
\frac{Q - Q_{4}}{Q_{3} - Q_{4}} \times U_{s,R,Q_{3}} + \frac{Q - Q_{3}}{Q_{4} - Q_{3}} \times U_{s,R,Q_{4}} & (Q_{3} \geq Q の場合)\\
\end{array}
\right.
$$

In [ ]:
# 暖冷房負荷計算時に想定した床の熱貫流率 (W/m2K) (4)
def get_U_s_vert(region, Q):
    """
    :param region: 省エネルギー地域区分
    :param Q: 当該住戸の熱損失係数 (W/m2K)
    :return: 暖冷房負荷計算時に想定した床の熱貫流率 (W/m2K)
    """
    if region == 8:
        return 2.223

    Q_0 = get_Q_j(region, 0)
    Q_1 = get_Q_j(region, 1)
    Q_2 = get_Q_j(region, 2)
    Q_3 = get_Q_j(region, 3)
    Q_4 = get_Q_j(region, 4)

    U_s_R_Q_0 = get_U_s_R_Q_j(region, 0)
    U_s_R_Q_1 = get_U_s_R_Q_j(region, 1)
    U_s_R_Q_2 = get_U_s_R_Q_j(region, 2)
    U_s_R_Q_3 = get_U_s_R_Q_j(region, 3)
    U_s_R_Q_4 = get_U_s_R_Q_j(region, 4)

    if Q > Q_1:
        U_s_vert = (Q - Q_1) / (Q_0 - Q_1) * U_s_R_Q_0 + (Q - Q_0) / (Q_1 - Q_0) * U_s_R_Q_1
    elif Q > Q_2:
        U_s_vert = (Q - Q_2) / (Q_1 - Q_2) * U_s_R_Q_1 + (Q - Q_1) / (Q_2 - Q_1) * U_s_R_Q_2
    elif Q > Q_3:
        U_s_vert = (Q - Q_3) / (Q_2 - Q_3) * U_s_R_Q_2 + (Q - Q_2) / (Q_3 - Q_2) * U_s_R_Q_3
    else:
        U_s_vert = (Q - Q_4) / (Q_3 - Q_4) * U_s_R_Q_3 + (Q - Q_3) / (Q_4 - Q_3) * U_s_R_Q_4

    # 2.223 を上回る場合は 2.223
    U_s_vert = min(2.223, U_s_vert)

    return U_s_vert

$U_{s,R,Q_{j}}$：断熱性能の区分$j$の熱損失係数$Q_{j}$における標準住戸の床の熱貫流率(W/(m<sup>2</sup>・K)) / Heat Transfer Coefficient of Standard Housing Floor at Heat Loss Coefficient $ Q_ {j} $ of Division $ j $ of Thermal Insulation Performance  
$U_{s,vert}$：暖冷房負荷計算時に想定した床の熱貫流率(W/(m<sup>2</sup>・K)) / Heat transmission coefficient of floor assumed when calculating heating and cooling load  
$Q$：当該住戸の熱損失係数(W/m<sup>2</sup>・K) / Heat loss coefficient of the dwelling unit concerned  
$Q_{j}$：断熱性能の区分$j$の熱損失係数(W/m<sup>2</sup>・K) / Heat loss coefficient of division $ j $ of thermal insulation performance  

<div style="text-align: center;font-weight: bold;">表　断熱性能の区分$j$の熱損失係数$Q_{j}$における標準住戸の床の熱貫流率$U_{s,R,Q_{j}}$(W/(m<sup>2</sup>・K))<br>Table: Heat Conductivity of Standard Housing Floor at Heat Loss Coefficient $ Q_ {j} $ of Division $ j $ of Thermal Insulation Performance</div>

<table>
<thead>
<tr>
<th rowspan="2" colspan="2"></th>
<th colspan="7">地域の区分<br>Segmentation of area</th>
</tr>
<tr>
<th>1</th>
<th>2</th>
<th>3</th>
<th>4</th>
<th>5</th>
<th>6</th>
<th>7</th>
</tr>
</thead>

<tbody>
<tr>
<th rowspan="5">断熱性能の区分<br>Classification of thermal insulation performance</th>
<th>0</th>
<td>2.223</td><td>2.223</td><td>2.223</td><td>2.223</td><td>2.223</td><td>2.223</td><td>2.223</td>
</tr>
<tr>
<th>1</th>
<td>0.342</td><td>0.342</td><td>0.964</td><td>0.949</td><td>1.208</td><td>1.208</td><td>2.223</td>
</tr>
<tr>
<th>2</th>
<td>0.427</td><td>0.427</td><td>0.791</td><td>0.825</td><td>1.360</td><td>1.360</td><td>1.633</td>
</tr>
<tr>
<th>3</th>
<td>0.319</td><td>0.319</td><td>0.359</td><td>0.564</td><td>0.554</td><td>0.554</td><td>0.554</td>
</tr>
<tr>
<th>4</th>
<td>0.272</td><td>0.272</td><td>0.272</td><td>0.373</td><td>0.373</td><td>0.373</td><td>0.373</td></tr>
</tbody>
</table>

In [ ]:
# 断熱性能の区分jの熱損失係数Q_jにおける標準住戸の床の熱貫流率 (W/m2K)
def get_U_s_R_Q_j(region, j):
    return table_e_3[j][region - 1]


# 表E.3 断熱性能の区分jの熱損失係数Qjにおける標準住戸の床の熱貫流率
table_e_3 = [
    (2.223, 2.223, 2.223, 2.223, 2.223, 2.223, 2.223, 2.223),
    (0.342, 0.342, 0.964, 0.949, 1.208, 1.208, 2.223, 2.223),
    (0.427, 0.427, 0.791, 0.825, 1.360, 1.360, 1.633, 2.223),
    (0.319, 0.319, 0.359, 0.564, 0.554, 0.554, 0.554, 2.223),
    (0.272, 0.272, 0.272, 0.373, 0.373, 0.373, 0.373, 0.373),
]


基礎の線熱貫流率$\varphi$が2.289 W/(m・K)を上回る場合には、2.289 W/(m・K)とする。また、8 地域においては、基礎の線熱貫流率$\varphi$は、当該住戸の熱損失係数$Q$に係わらず2.289 W/(m・K)とする。 

If the linear heat transfer coefficient of the foundation $ \varphi $ exceeds 2.289 W/(m・K), set it to 2.289 W/(m・K). In addition, in the 8 areas, the linear heat transfer coefficient $ \varphi $ of the foundation is set to 2.289W/(m・K) regardless of the heat loss coefficient $ Q $ of the relevant dwelling unit.

$$
\large
\varphi = \left\{
\begin{array}{ll}
\frac{Q - Q_{1}}{Q_{0} - Q_{1}} \times \varphi_{Q_{0}} + \frac{Q - Q_{0}}{Q_{1} - Q_{0}} \times \varphi_{Q_{1}} & (Q > Q_{1} の場合)\\
\frac{Q - Q_{2}}{Q_{1} - Q_{2}} \times \varphi_{Q_{1}} + \frac{Q - Q_{1}}{Q_{2} - Q_{1}} \times \varphi_{Q_{2}} & (Q_{1} \geq Q > Q_{2} の場合)\\
\frac{Q - Q_{3}}{Q_{2} - Q_{3}} \times \varphi_{Q_{2}} + \frac{Q - Q_{2}}{Q_{3} - Q_{2}} \times \varphi_{Q_{3}} & (Q_{2} \geq Q > Q_{3} の場合)\\
\frac{Q - Q_{4}}{Q_{3} - Q_{4}} \times \varphi_{Q_{3}} + \frac{Q - Q_{3}}{Q_{4} - Q_{3}} \times \varphi_{Q_{4}} & (Q_{3} \geq Q の場合)\\
\end{array}
\right.
$$

In [ ]:
# 基礎の線熱貫流率 (W/mK) (5)
def get_phi(region, Q):
    """
    :param region: 省エネルギー地域区分
    :param Q: 当該住戸の熱損失係数 (W/m2K)
    :return: 基礎の線熱貫流率 (W/mK)
    """
    # 8地域においては、当該住戸の熱損失係数に瓦らず　2.289 W/mK
    if region == 8:
        return 2.289

    Q_0 = get_Q_j(region, 0)
    Q_1 = get_Q_j(region, 1)
    Q_2 = get_Q_j(region, 2)
    Q_3 = get_Q_j(region, 3)
    Q_4 = get_Q_j(region, 4)

    phi_Q_0 = get_phi_Q_j(region, 0)
    phi_Q_1 = get_phi_Q_j(region, 1)
    phi_Q_2 = get_phi_Q_j(region, 2)
    phi_Q_3 = get_phi_Q_j(region, 3)
    phi_Q_4 = get_phi_Q_j(region, 4)

    if Q > Q_1:
        phi = (Q - Q_1) / (Q_0 - Q_1) * phi_Q_0 + (Q - Q_0) / (Q_1 - Q_0) * phi_Q_1
    elif Q > Q_2:
        phi = (Q - Q_2) / (Q_1 - Q_2) * phi_Q_1 + (Q - Q_1) / (Q_2 - Q_1) * phi_Q_2
    elif Q > Q_3:
        phi = (Q - Q_3) / (Q_2 - Q_3) * phi_Q_2 + (Q - Q_2) / (Q_3 - Q_2) * phi_Q_3
    else:
        phi = (Q - Q_4) / (Q_3 - Q_4) * phi_Q_3 + (Q - Q_3) / (Q_4 - Q_3) * phi_Q_4

    # 2.289 を上回る場合は 2.289
    phi = min(2.289, phi)

    return phi

$\varphi$：基礎の線熱貫流率(W/(m・K)) / Line heat penetration coefficient of foundation  
$\varphi_{Q_{j}}$：断熱性能の区分$j$の熱損失係数$Q_{j}$における基礎の線熱貫流率(W/(m・K)) / Linear heat transmission coefficient of foundation at heat loss coefficient $ Q_ {j} $ of division $ j $ of adiabatic performance  
$Q$：当該住戸の熱損失係数(W/m<sup>2</sup>・K) / Heat loss coefficient of the dwelling unit concerned  
$Q_{j}$：断熱性能の区分$j$の熱損失係数(W/m<sup>2</sup>・K) / Heat loss coefficient of division $ j $ of thermal insulation performance  

<div style="text-align: center;font-weight: bold;">表　断熱性能の区分$j$の熱損失係数$Q_{j}$における基礎の線熱貫流率$\varphi_{Q_{j}}$ (W/(m・K))<br>Table: Adiabatic performance section $ j $ heat loss coefficient of foundation $ Q_ {j} $ at linear heat transmission coefficient $\varphi_{Q_{j}}$</div>

<table>
<thead>
<tr>
<th rowspan="2" colspan="2"></th>
<th colspan="7">地域の区分<br>Segmentation of area</th>
</tr>
<tr>
<th>1</th>
<th>2</th>
<th>3</th>
<th>4</th>
<th>5</th>
<th>6</th>
<th>7</th>
</tr>
</thead>

<tbody>
<tr>
<th rowspan="5">断熱性能の区分<br>Classification of thermal insulation performance</th>
<th>0</th>
<td>2.289</td><td>2.289</td><td>2.289</td><td>2.289</td><td>2.289</td><td>2.289</td><td>2.289</td>
</tr>
<tr>
<th>1</th>
<td>0.635</td><td>0.635</td><td>1.517</td><td>1.504</td><td>1.872</td><td>1.872</td><td>2.289</td>
</tr>
<tr>
<th>2</th>
<td>0.675</td><td>0.675</td><td>1.272</td><td>1.329</td><td>2.087</td><td>2.087</td><td>2.289</td>
</tr>
<tr>
<th>3</th>
<td>0.516</td><td>0.516</td><td>0.573</td><td>0.877</td><td>0.863</td><td>0.863</td><td>0.863</td>
</tr>
<tr>
<th>4</th>
<td>0.450</td><td>0.450</td><td>0.450</td><td>0.601</td><td>0.601</td><td>0.601</td><td>0.601</td>
</tbody>
</table>

In [ ]:
# 断熱性能の区分jの熱損失係数Q_jにおける基礎の線熱貫流率 (W/mK)
def get_phi_Q_j(region, j):
    """
    :param region: 省エネルギー地域区分
    :param j: 断熱性能の区分
    :return: 断熱性能の区分jの熱損失係数Q_jにおける基礎の線熱貫流率 (W/mK)
    """
    return table_e_4[j][region - 1]


# 表E.4 断熱性能の区分jの計算対象住戸の基礎の仮想線熱貫流率
table_e_4 = [
    (2.289, 2.289, 2.289, 2.289, 2.289, 2.289, 2.289, 2.289),
    (0.635, 0.635, 1.517, 1.504, 1.872, 1.872, 2.289, 2.289),
    (0.675, 0.675, 1.272, 1.329, 2.087, 2.087, 2.289, 2.289),
    (0.516, 0.516, 0.573, 0.877, 0.863, 0.863, 0.863, 2.289),
    (0.450, 0.450, 0.450, 0.601, 0.601, 0.601, 0.601, 2.289),
]

<div style="text-align: center;font-weight: bold;">表　断熱性能の区分${j}$の熱損失係数$Q_{j}$ (W/(m<sup>2</sup>・K))<br>Table: Adiabatic performance division $ {j} $ heat loss coefficient $ Q_ {j} $</div>

<table>
<thead>
<tr>
<th rowspan="2" colspan="2"></th>
<th colspan="7">地域の区分<br>Segmentation of area</th>
</tr>
<tr>
<th>1</th>
<th>2</th>
<th>3</th>
<th>4</th>
<th>5</th>
<th>6</th>
<th>7</th>
</tr>
</thead>

<tbody>
<tr>
<th rowspan="5">断熱性能の区分<br>Classification of thermal insulation performance</th>
<th>0</th>
<td>8.26</td><td>8.26</td><td>8.26</td><td>8.49</td><td>8.49</td><td>8.49</td><td>8.49</td>
</tr>
<tr>
<th>1</th>
<td>2.8</td><td>2.8</td><td>4.0</td><td>4.7</td><td>5.19</td><td>5.19</td><td>8.27</td>
</tr>
<tr>
<th>2</th>
<td>1.8</td><td>1.8</td><td>2.7</td><td>3.3</td><td>4.2</td><td>4.2</td><td>4.59</td>
</tr>
<tr>
<th>3</th>
<td>1.6</td><td>1.6</td><td>1.9</td><td>2.4</td><td>2.7</td><td>2.7</td><td>2.7</td>
</tr>
<tr>
<th>4</th>
<td>1.4</td><td>1.4</td><td>1.4</td><td>1.9</td><td>1.9</td><td>1.9</td><td>1.9</td>
</tbody>
</table>

In [ ]:
# 断熱性能の区分jの熱損失係数 Q_j (W/m2K)
def get_Q_j(region, j):
    """
    :param region: 省エネルギー地域区分
    :param j: 断熱性能の区分
    :return: 断熱性能の区分jの熱損失係数 Q_j (W/m2K)
    """
    return table_e_5[j][region - 1]


# 表E.5 断熱性能の区分jの熱損失係数
table_e_5 = [
    (8.26, 8.26, 8.26, 8.49, 8.49, 8.49, 8.49, 8.49),
    (2.8, 2.8, 4.0, 4.7, 5.19, 5.19, 8.27, 8.27),
    (1.8, 1.8, 2.7, 3.3, 4.2, 4.2, 4.59, 8.01),
    (1.6, 1.6, 1.9, 2.4, 2.7, 2.7, 2.7, 2.7),
    (1.4, 1.4, 1.4, 1.9, 1.9, 1.9, 1.9, 3.7),
]

### E.4 床下空間に接する床の面積及び床下空間の基礎外周長さ / The area of the floor in contact with the underfloor space and the base circumference length of the underfloor space

$$\large
\begin{equation*}
L_{uf} = r_{a} \times 4 \times \sqrt{A_{s,ufvnt,A}}
\end{equation*}
$$

In [ ]:
# 当該住戸の外気を導入する床下空間の基礎外周長さ (6)
def get_L_uf(A_s_ufvnt_A):
    """
    :param A_s_ufvnt_A: 当該住戸の外気を導入する床下空間に接する床の面積の合計 (m2)
    :return: 当該住戸の外気を導入する床下空間の基礎外周長さ (m)
    """
    # 床下の平面形状(アスペクト比等)を考慮した係数(無次元)
    r_a = get_r_a()

    # 当該住戸の外気を導入する床下空間の基礎外周長さ (6)
    L_uf = r_a * 4 * sqrt(A_s_ufvnt_A)

    return L_uf

# 床下の平面形状(アスペクト比等)を考慮した係数(無次元)
def get_r_a():
    return 1.02

$L_{uf}$：当該住戸の外気を導入する床下空間の基礎外周長さ(m) / Base perimeter length of underfloor space which introduces the open air of the dwelling unit concerned  
$r_{a}$：床下の平面形状（アスペクト比等）を考慮した係数（無次元）。$r_{a} = 1.02$ とする。 / A factor (dimensionless) that takes into account the planar shape (such as aspect ratio) under the floor. Let $ r_ {a} = 1.02 $.   
$A_{s,ufvnt,A}$：当該住戸の空気を供給する床下空間に接する床の面積の合計(m<sup>2</sup>) /  Total floor area in contact with the floor space that supplies the air of the dwelling unit  

$$\large
\begin{equation*}
A_{s,ufvnt,i} = r_{A,uf,i} \times A_{HCZ,i} \times r_{A,ufvnt}
\end{equation*}
$$

$$\large
\begin{equation*}
A_{s,ufvnt,A} = \sum_{i=1}^{12} A_{s,ufvnt,i}
\end{equation*}
$$

In [ ]:
# 当該住戸の暖冷房区画iの空気を供給する床下空間に接する床の面積 (m2) (7)
def get_A_s_ufvnt_i(i, r_A_ufvnt, A_A, A_MR, A_OR):
    """
    :param i: 当該住戸の暖冷房区画の番号
    :param r_A_ufvnt: 当該住戸において、床下空間全体の面積に対する空気を供給する床下空間の面積の比 (-)
    :return: 当該住戸の暖冷房区画iの空気を供給する床下空間に接する床の面積 (m2)
    """
    # 暖冷房区画iの床面積のうち床下空間に接する床面積の割合 (-)
    r_A_uf_i = get_r_A_uf_i(i)

    from section3_1 import get_A_HCZ_i
    A_HCZ_i = get_A_HCZ_i(i, A_A, A_MR, A_OR)

    # 当該住戸の暖冷房区画iの空気を供給する床下空間に接する床の面積 (m2) (7)
    A_s_ufvnt_i = r_A_uf_i * A_HCZ_i * r_A_ufvnt

    return A_s_ufvnt_i


In [ ]:

# 当該住戸の空気を供給する床下空間に接する床の面積の合計 (m2) (8)
def get_A_s_ufvnt_A(r_A_ufvnt, A_A, A_MR, A_OR):
    """
    :param r_A_ufvnt: 当該住戸において、床下空間全体の面積に対する空気を供給する床下空間の面積の比 (-)
    :return: 当該住戸の空気を供給する床下空間に接する床の面積の合計 (m2)
    """
    # 当該住戸の空気を供給する床下空間に接する床の面積の合計 (m2) (8)
    A_s_ufvnt_A = sum([get_A_s_ufvnt_i(i, r_A_ufvnt, A_A, A_MR, A_OR) for i in range(1, 13)])

    return A_s_ufvnt_A


$A_{s,ufvnt,A}$：当該住戸の空気を供給する床下空間に接する床の面積の合計(m<sup>2</sup>) / Total floor area in contact with the floor space that supplies the air of the dwelling unit  
$A_{s,ufvnt,i}$：当該住戸の暖冷房区画$i$の空気を供給する床下空間に接する床の面積(m<sup>2</sup>) / Floor area in contact with the underfloor space that supplies air for heating / cooling section $ i $ of the relevant dwelling unit  
$A_{HCZ,i}$：暖冷房区画$i$の床面積(m<sup>2</sup>) / Floor area of heating / cooling section $ i $  
$r_{A,ufvnt}$：当該住戸において、床下空間全体の面積に対する空気を供給する床下空間の面積の比(-) / The ratio of the area of the underfloor space supplying air to the area of the entire underfloor space in the dwelling unit   
$r_{A,uf,i}$：当該住戸の暖冷房区画$i$の床面積のうち床下空間に接する床面積の割合(-) / Ratio of floor area contacting floor space to floor area of heating and cooling section $ i $ of the relevant dwelling unit   

<div style="text-align: center;font-weight: bold;">表　暖冷房区画$i$の床面積のうち床下空間に接する床面積の割合$r_{A,uf,i}$<br>Table: Ratio of floor area that touches floor space to floor space of heating / cooling compartment $ i $ $ r_ {A, uf, i} $</div>

|暖冷房区画$i$の番号<br>Number of heating / cooling section $ i $|室名<br>Room name|床下空間に接する床面積の割合$r_{A,uf,i}$（-）<br>Percentage of floor area in contact with the floor space $r_{A,uf,i}$|
|-|-|-|
|1|LDK|1.0|
|2|和室<br>Japanese room|1.0|
|3|主寝室<br>Master bedroom|0|
|4|子供室1<br>Children's room 1|0|
|5|子供室2<br>Children's room 2|0|
|6|浴室<br>Bathroom|1.0|
|7|1F便所<br>1F toilet|1.0|
|8|洗面所<br>Washroom|1.0|
|9|ホール<br>Hole|$\frac{10.76}{13.25}$|
|10|クローゼット<br>Closet|0|
|11|2Fホール<br>2F Hole|0|
|12|2F便所<br>2F toilet|0|


In [ ]:
# 暖冷房区画iの床面積のうち床下空間に接する床面積の割合 (-)
def get_r_A_uf_i(i):
    return table_e_6[i - 1]


# 表 E.6 暖冷房区画iの床面積のうち床下空間に接する床面積の割合
table_e_6 = (1.0, 1.0, 0, 0, 0, 1.0, 1.0, 1.0, 10.76 / 13.25, 0, 0, 0)

### E.5 地盤またはそれを覆う基礎 / Ground or foundation covering it

$$\large
\begin{equation*}
   \theta_{g,surf,d,t} = \frac{\frac{\phi_{A,0}}{R_g} \theta_{uf,d,t} + \sum_{m=1}^{M} \theta'_{g,surf,A,m,d,t} + \theta_{g,avg}}{1 + \frac{\phi_{A,0}}{R_g}}
\end{equation*}
$$

$M$：指数項の総数(-) / Total number of exponential terms  
$R_g$：地盤またはそれを覆う基礎の表面熱伝達抵抗((m<sup>2</sup>・K)/W) /   
$\theta_{g,avg}$：地盤の不易層温度(℃) / Impossible layer temperature of the ground  
$\theta_{uf,d,t}$：日付$d$の時刻$t$における当該住戸の床下温度(℃) / Under floor temperature of the dwelling unit at time $ t $ of date $ d $  
$\theta_{g,surf,d,t}$：日付$d$の時刻$t$における地盤またはそれを覆う基礎の表面温度(℃) / Surface temperature of the ground or foundation covering it at time $ t $ of date $ d $  
$\theta'_{g,surf,A,m,d,t}$：日付$d$の時刻$t$における指数項$m$の吸熱応答の項別成分(℃) / Term component of endothermic response of exponent term $ m $ at time $ t $ of date $ d $  
$\phi_{A,0}$：吸熱応答係数の初項((m<sup>2</sup>・K)/W) / First term of endothermic response coefficient  

指数項の総数$M$は、10とする。地盤またはそれを覆う基礎の表面熱伝達抵抗$R_g$は、0.15 (m<sup>2</sup>・K)/W とする。吸熱応答係数の初項$\phi_{A,0}$は、0.025504994 (m<sup>2</sup>・K)/W とする。

The total number of exponent terms $ M $ is 10. The surface heat transfer resistance $ R_g $ of the ground or the foundation covering it is (m<sup>2</sup>・K)/W . The first term $ \phi_ {A, 0} $ of the endothermic response coefficient is 0.025504994  (m<sup>2</sup>・K)/W.

$$\large
\begin{equation*}
   \theta_{g,avg} = \frac{\sum_{d=1}^{365} \sum_{t=0}^{23} \theta_{ex,d,t}}{8760}
\end{equation*}
$$

In [ ]:
# 地盤の不易層温度 (℃) (10)
def get_Theta_g_avg(Theta_ex_d_t):
    """
    :param Theta_ex_d_t: 外気温度 (℃)
    :return: 地盤の不易層温度 (℃)
    """
    # 地盤の不易層温度は年間平均気温に等しいとする
    return np.average(Theta_ex_d_t)

$\theta_{ex,d,t}$：日付$d$の時刻$t$における外気温度(℃) / Outside temperature at time $ t $ of date $ d $  
$\theta_{g,avg}$：地盤の不易層温度(℃) / Impossible layer temperature of the ground  

$$\large
\begin{equation*}
    \theta'_{g,surf,A,m,d,t} = \phi_{1,A,m} q_{g,d,t-1} + r_{m} \theta'_{g,surf,A,m,d,t-1}
\end{equation*}
$$

In [ ]:
# 指数項mにおける公比r_m
def get_r_m(m):
    return table_e_8[m - 1][0]


$q_{g,d,t-1}$：日付$d$の時刻$t-1$における地盤またはそれを覆う基礎の表面熱流(W/m<sup>2</sup>) / Surface heat flow of the ground or foundation covering it at time $ t-1 $ at date $ d $   
$r_{m}$：指数項$m$における公比(-) / Common ratio in the index term $ m $  
$\theta'_{g,surf,A,m,d,t-1}$：日付$d$の時刻$t-1$における指数項$m$の吸熱応答の項別成分(℃) / Term component of endothermic response of exponent term $ m $ at time $ t-1 $ at date $ d $  
$\phi_{1,A,m}$：指数項$m$における吸熱応答係数((m<sup>2</sup>・K)/W) / Endothermic response coefficient at exponential term $ m $  

<div style="text-align: center;font-weight: bold;">表　指数項$m$における公比$r_{m}$及び吸熱応答係数$\phi_{1,A,m}$<br>Table: $ r_ {m} $ common in $ m $ exponential term and endothermic response coefficient $ \ phi_ {1, A, m} $</div>

|指数項$m$<br>Exponential term|公比$r_{m}$(-)<br>Common ratio|吸熱応答係数$\phi_{1,A,m}$((m<sup>2</sup>・K)/W)<br>Endothermic response coefficient|
|-|-|-|
|1|0.999996185007277|-0.0000000373932617900396|
|2|0.999984740116433|0.0000007335783188647870|
|3|0.999938961862903|-0.0000099402124533592900|
|4|0.999755929789981|0.0005842990301842850000|
|5|0.999023876718580|0.0003844348238087120000|
|6|0.996101618495491|0.0006186789168868810000|
|7|0.984491514535530|0.0027429049659913000000|
|8|0.939413062813476|0.0008772601274016750000|
|9|0.778800783071405|0.0116100392142522000000|
|10|0.367879441171442|0.0015655689737126900000|


In [ ]:
# 指数項mにおける吸熱応答係数phi_1_A_m
def get_phi_1_A_m(m):
    return table_e_8[m - 1][1]


# 表 E.7 指数項mにおける公比r_m及び吸熱応答係数phi_1_A_m
table_e_8 = [
    (0.999996185007277, -0.0000000373932617900396),
    (0.999984740116433, 0.0000007335783188647870),
    (0.999938961862903, -0.0000099402124533592900),
    (0.999755929789981, 0.0005842990301842850000),
    (0.999023876718580, 0.00038443482380871200009),
    (0.996101618495491, 0.0006186789168868810000),
    (0.984491514535530, 0.0027429049659913000000),
    (0.939413062813476, 0.0008772601274016750000),
    (0.778800783071405, 0.0116100392142522000000),
    (0.367879441171442, 0.0015655689737126900000)
]

$$\large
\begin{equation*}
    q_{g,d,t-1} = \frac{1}{R_g} (\theta_{uf,d,t-1} - \theta_{g,surf,d,t-1})
\end{equation*}
$$

$q_{g,d,t-1}$：日付$d$の時刻$t-1$における地盤またはそれを覆う基礎の表面熱流(W/m<sup>2</sup>) / Surface heat flow of the ground or foundation covering it at time $ t-1 $ at date $ d $   
$R_g$：地盤またはそれを覆う基礎の表面熱伝達抵抗((m<sup>2</sup>・K)/W) / Surface heat transfer resistance of ground or foundation covering it  
$\theta_{uf,d,t-1}$：日付$d$の時刻$t-1$における当該住戸の床下温度(℃) / Under floor temperature of the relevant dwelling unit at time $ t-1 $ at date $ d $  
$\theta_{g,surf,d,t-1}$：日付$d$の時刻$t-1$における地盤またはそれを覆う基礎の表面温度(℃) / Surface temperature of the ground or foundation covering it at time $ t-1 $ at date $ d $  

地盤またはそれを覆う基礎の表面熱伝達抵抗$R_g$は、0.15 (m<sup>2</sup>・K)/W とする。 

The surface heat transfer resistance $ R_g $ of the ground or the foundation covering it is 0.15 (m<sup>2</sup>・K)/W.

日付$d$の時刻$t$が 1 月 1 日 0 時の場合、1 時刻前における当該住戸の床下温度$\theta_{uf,d,t-1}$、地盤またはそれを覆う基礎の表面温度$\theta_{g,surf,d,t-1}$及び指数項$m$の吸熱応答の項別成分$\theta'_{g,surf,A,m,d,t-1}$ には、地盤またはそれを覆う基礎の表面温度について助走計算を行うことにより得られる最終値を用いることとする。 

If the time $ t $ on the date $ d $ is 0 January 1, the floor temperature $ \theta_ {uf, d, t-1} $ of the dwelling unit at 1 hour ago, the ground or the surface of the foundation that covers it Term component $ \theta '_ {g, surf, A, m, d, t-1} $ of endothermic response of temperature $ \theta_ {g, surf, d, t-1} $ and exponent term $ m $ In the above, we use the final value obtained by performing an approach calculation for the surface temperature of the ground or the foundation that covers it.

助走計算では、計算タイムステップを 1 時間、計算期間を 1 月 1 日 0 時から 12 月 31 日 23 時までの 1 年間とする。

In the approach calculation, the calculation time step is 1 hour, and the calculation period is 1 year from January 1st to 23rd December 31st.

床下空間が断熱区画内である場合： 

If the underfloor space is in an insulated compartment:

$\theta_{ex,d,t} < \theta_{in,H}$ の場合： 

In the case of $\theta_{ex,d,t} < \theta_{in,H}$:

$$\large
\begin{equation*}
\theta_{uf,d,t} = \theta_{in,H}
\end{equation*}
$$

$\theta_{in,C} < \theta_{ex,d,t}$ の場合： 

In the case of $\theta_{in,C} < \theta_{ex,d,t}$:

$$\large
\begin{equation*}
\theta_{uf,d,t} = \theta_{in,C}
\end{equation*}
$$

$$\large
\begin{equation*}
\theta_{uf,d,t} = \theta_{ex,d,t}
\end{equation*}
$$

床下空間が断熱区画外である場合：

If the underfloor space is outside the insulation compartment:

$\theta_{ex,d,t} < \theta_{in,H}$ の場合： 

In the case of $\theta_{ex,d,t} < \theta_{in,H}$:

$$\large
\begin{equation*}
\theta_{uf,d,t} = \theta_{ex,d,t} \times H_{base,IS} + \theta_{in,H} \times (1 - H_{floor})
\end{equation*}
$$

$\theta_{in,C} < \theta_{ex,d,t}$ の場合： 

In the case of $\theta_{in,C} < \theta_{ex,d,t}$:

$$\large
\begin{equation*}
\theta_{uf,d,t} = \theta_{ex,d,t} \times H_{base,IS} + \theta_{in,C} \times (1 - H_{floor})
\end{equation*}
$$

$\theta_{in,H} \leq \theta_{ex,d,t} \leq \theta_{in,C}$ の場合： 

In the case of $\theta_{in,H} \leq \theta_{ex,d,t} \leq \theta_{in,C}$:

$$\large
\begin{equation*}
\theta_{uf,d,t} = \theta_{ex,d,t}
\end{equation*}
$$

$H_{floor}$：床の温度差係数(-) / Floor temperature difference coefficient  
$\theta_{ex,d,t}$：日付$d$の時刻$t$における外気温度(℃) / Outside temperature at time $ t $ of date $ d $  
$\theta_{in,C}$：冷房時の室温(℃) / Room temperature at cooling  
$\theta_{in,H}$：暖房時の室温(℃) / Room temperature when heating  
$\theta_{uf,d,t}$：日付$d$の時刻$t$における当該住戸の床下温度(℃) / Under floor temperature of the dwelling unit at time $ t $ of date $ d $  

床の温度差係数$H_{floor}$は、0.7 とする。冷房時の室温$\theta_{in,C}$は、27℃とする。暖房時の室温$\theta_{in,H}$は、20℃とする。また、日付$d$の時刻$t$が 1 月 1 日 0 時の場合、1 時刻前における当該住戸の床下温度$\theta_{uf,d,t-1}$、地盤またはそれを覆う基礎の表面温度$\theta_{g,surf,d,t-1}$及び指数項$m$の吸熱応答の項別成分$\theta'_{g,surf,A,m,d,t-1}$ は、いずれもゼロとする。 

The floor temperature difference coefficient $ H_ {floor} $ is 0.7. The room temperature $ \theta_ {in, C} $ at the time of cooling is 27 ° C. The room temperature $ \theta_ {in, H} $ at heating is assumed to be 20 ° C. Also, if the time $ t $ on the date $ d $ is 0 January 1, the floor temperature $ \theta_ {uf, d, t-1} $ of that dwelling unit one hour earlier, the ground or the foundation that covers it Term components of endothermic response $ \ theta _ {g, surf, d, t-1} $ of $ and $ m $ exponential term $ \theta'_ {g, surf, A, m, d, t-1 } $ Is all zero.

In [ ]:
# <<助走計算>> 当該住戸の床下温度 (℃) (13)
def get_Theta_uf_d_t_runup(underfloor_insulation, Theta_ex_d_t):
    """
    :param underfloor_insulation: 床下空間が断熱空間内である場合はTrue
    :param Theta_ex_d_t: 外気温度 (℃)
    :return: 当該住戸の床下温度 (℃)
    """
    # 床の温度差係数 (-)
    H_floor = 0.7

    # TODO: H_base_IS は仕様書においては未定義
    H_base_IS = 0.7

    # 冷房時の室温 (℃)
    Theta_in_C = 27.0

    # 暖房時の室温 (℃)
    Theta_in_H = 20.0

    # 計算領域の確保
    Theta_uf_d_t = np.zeros(24 * 365)

    if underfloor_insulation:
        # 床下空間が断熱区間内である場合
        # (13-1a) Theta_ex_d_t < Theta_in_H の場合
        f1a = Theta_ex_d_t < Theta_in_H
        Theta_uf_d_t[f1a] = Theta_in_H

        # (13-2a) Theta_in_C < Theta_ex_d_t の場合
        f2a = Theta_in_C < Theta_ex_d_t
        Theta_uf_d_t[f2a] = Theta_in_C

        # (13-3a) Theta_in_H <= Theta_ex_d_t <= Theta_in_C の場合
        f3a = np.logical_and(Theta_in_H <= Theta_ex_d_t, Theta_ex_d_t <= Theta_in_C)
        Theta_uf_d_t[f3a] = Theta_ex_d_t[f3a]
    else:
        # 床下空間が断熱区間外である場合
        # (13-1b) Theta_ex_d_t < Theta_in_H の場合
        f1b = Theta_ex_d_t < Theta_in_H
        Theta_uf_d_t[f1b] = Theta_ex_d_t[f1b] * H_base_IS + Theta_in_H * (1 - H_floor)

        # (13-2b) Theta_in_C < Theta_ex_d_t の場合
        f2b = Theta_in_C < Theta_ex_d_t
        Theta_uf_d_t[f2b] = Theta_ex_d_t[f2b] * H_base_IS + Theta_in_C * (1 - H_floor)

        # (13-3b) Theta_in_H <= Theta_ex_d_t <= Theta_in_C の場合
        f3b = np.logical_and(Theta_in_H <= Theta_ex_d_t, Theta_ex_d_t <= Theta_in_C)
        Theta_uf_d_t[f3b] = Theta_ex_d_t[f3b]

    return Theta_uf_d_t